# GoogLeNet

In [1]:
import keras
import numpy as np
from keras.datasets import cifar10, cifar100
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Add, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras.initializers import glorot_uniform
from keras.regularizers import l2
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras import optimizers

Using TensorFlow backend.


In [2]:
c10_class_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
c100_class_name = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 
                   'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 
                   'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 
                   'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse','mushroom', 'oak_tree', 'orange', 'orchid', 
                   'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 
                   'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 
                   'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 
                   'willow_tree', 'wolf', 'woman', 'worm']

In [3]:
def inception_module(X, f1, f2_in, f2_out, f3_in, f3_out, f4_out, stage, block):
    
    name_sb = str(stage) + str(block)
    
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu', name = 'conv_tower1_' + name_sb)(X)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu', name = 'conv_tower2_1_' + name_sb)(X)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu', name = 'conv_tower2_2' + name_sb)(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu', name = 'conv_tower3_1_' + name_sb)(X)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu', name = 'conv_tower3_2_' + name_sb)(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same', name = 'mpool_tower4_' + name_sb)(X)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu', name = 'conv_tower4_' + name_sb)(pool)
    
    # concatenate filters
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    
    return layer_out

In [4]:
def auxiliary(x, classes, name=None):
    
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=classes, activation='softmax', name=name)(layer)
    
    return layer

In [5]:
def googlenet(input_shape, classes):
    
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (3, 3), strides = (2, 2), name = 'conv_pre_inception_1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_pre_inception_1')(X)
    
    # Stage 2
    X = Conv2D(192, (3, 3), strides = (1, 1), name = 'conv_pre_inception_2', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_2')(X)
    X = Activation('relu')(X)
    X = Conv2D(192, (3, 3), strides = (1, 1), name = 'conv_pre_inception_3', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_pre_inception_2')(X)
    
    # Stage 3
    X = inception_module(X, 64, 96, 128, 16, 32, 32, 3, 'a')
    X = inception_module(X, 128, 128, 192, 32, 96, 64, 3, 'b')
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_inception_3')(X)
    
    # Stage 4
    X = inception_module(X, 192, 96, 208, 16, 48, 64, 4, 'a')
    aux1  = auxiliary(X, classes, name='aux1')
    X = inception_module(X, 160, 112, 224, 24, 64, 64, 4, 'b')
    X = inception_module(X, 128, 128, 256, 24, 24, 64, 4, 'c')
    X = inception_module(X, 112, 144, 288, 32, 64, 64, 4, 'd')
    aux2  = auxiliary(X, classes, name='aux2')
    X = inception_module(X, 256, 120, 320, 32, 128, 128, 4, 'e')
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_inception_4')(X)
    
    # Stage 5
    X = inception_module(X, 256, 160, 320, 32, 128, 128, 5, 'a')
    X = inception_module(X, 384, 192, 384, 48, 128, 128, 5, 'b')
    X = AveragePooling2D((7, 7), strides=(1, 1), name = 'apool_inception_5')(X)
    
    # Stage 6
    X = Dropout(0.4)(X)
    X = Flatten()(X)
    X = Dense(1000, activation='relu', kernel_initializer = glorot_uniform(seed=0))(X)
    main = Dense(classes, activation='softmax', name='main', kernel_initializer = glorot_uniform(seed=0))(X)
    
    aux1 = keras.layers.Lambda(lambda x: x * 0.3)(aux1)                                         
    aux2 = keras.layers.Lambda(lambda x: x * 0.3)(aux2)                                         
    totalloss = keras.layers.Add()([aux1, aux2, main])                     
    
    inceptionv1 = Model(inputs = X_input, outputs = [totalloss], name='InceptionV1')            
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    inceptionv1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return inceptionv1

### CIFAR - 10

In [6]:
(c10_x_train_orig, c10_y_train_orig), (c10_x_test_orig, c10_y_test_orig) = cifar10.load_data()

# Pre-processing X
c10_x_train = c10_x_train_orig.astype('float32')
c10_x_test = c10_x_test_orig.astype('float32')
c10_x_train = c10_x_train/255
c10_x_test = c10_x_test/255

# Hyper-parameters
c10_epochs = 50
c10_batch_size = 128
c10_num_classes = 10
c10_input_shape = c10_x_train.shape[1:]

# Pre-processing Y
c10_y_train = keras.utils.to_categorical(c10_y_train_orig, c10_num_classes)
c10_y_test = keras.utils.to_categorical(c10_y_test_orig, c10_num_classes)

print(c10_x_train.shape, c10_x_test.shape, c10_y_train.shape, c10_y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


In [7]:
c10_googlenet = googlenet(c10_input_shape, c10_num_classes)
c10_googlenet.summary()

Model: "InceptionV1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_pre_inception_1 (Conv2D)   (None, 18, 18, 32)   896         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_pre_inception_1 (BatchNormal (None, 18, 18, 32)   128         conv_pre_inception_1[0][0]       
________________________________________________________________________________________

In [8]:
c10_googlenet.fit(c10_x_train, c10_y_train, validation_data = (c10_x_test, c10_y_test), epochs = c10_epochs, batch_size = c10_batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 94s 2ms/step - loss: 1.5644 - accuracy: 0.4086 - val_loss: 2.1582 - val_accuracy: 0.2336
Epoch 2/50
50000/50000 [==============================] - 83s 2ms/step - loss: 1.1712 - accuracy: 0.5754 - val_loss: 1.1011 - val_accuracy: 0.6033
Epoch 3/50
50000/50000 [==============================] - 83s 2ms/step - loss: 0.9966 - accuracy: 0.6452 - val_loss: 1.0458 - val_accuracy: 0.6180
Epoch 4/50
50000/50000 [==============================] - 83s 2ms/step - loss: 0.8869 - accuracy: 0.6860 - val_loss: 0.9362 - val_accuracy: 0.6564
Epoch 5/50
50000/50000 [==============================] - 83s 2ms/step - loss: 0.7800 - accuracy: 0.7270 - val_loss: 0.8768 - val_accuracy: 0.6913
Epoch 6/50
50000/50000 [==============================] - 83s 2ms/step - loss: 0.6936 - accuracy: 0.7601 - val_loss: 0.8487 - val_accuracy: 0.7031
Epoch 7/50
50000/50000 [==============================] - 83s 2ms/st

In [9]:
scores = c10_googlenet.evaluate(c10_x_test, c10_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 666us/step
Test loss: 1.2876167963027954
Test accuracy: 0.7613000273704529


In [12]:
predictions = {}
for i in range(c10_num_classes):
    index = np.where(c10_y_test_orig == i)
    slice_x_test = c10_x_test[index[0]]
    slice_y_test = c10_y_test[index[0]]
    scores = c10_googlenet.evaluate(slice_x_test, slice_y_test, verbose = 0)
    print('Class: ' + c10_class_name[i] + ' Test accuracy: ' + str(scores[1]))
    predictions[c10_class_name[i]] = str(scores[1])

Class: airplane Test accuracy: 0.7089999914169312
Class: automobile Test accuracy: 0.8939999938011169
Class: bird Test accuracy: 0.6690000295639038
Class: cat Test accuracy: 0.5450000166893005
Class: deer Test accuracy: 0.7229999899864197
Class: dog Test accuracy: 0.8339999914169312
Class: frog Test accuracy: 0.7699999809265137
Class: horse Test accuracy: 0.7310000061988831
Class: ship Test accuracy: 0.8330000042915344
Class: truck Test accuracy: 0.9049999713897705


### CIFAR - 100

In [6]:
# Load data
(c100_x_train_orig, c100_y_train_orig), (c100_x_test_orig, c100_y_test_orig) = cifar100.load_data()

# Pre-processing X
c100_x_train = c100_x_train_orig.astype('float32')
c100_x_test = c100_x_test_orig.astype('float32')
c100_x_train = c100_x_train/255
c100_x_test = c100_x_test/255

# Hyper-parameters
c100_epochs = 50
c100_batch_size = 128
c100_num_classes = 100
c100_input_shape = c100_x_train.shape[1:]

# Pre-processing Y
c100_y_train = keras.utils.to_categorical(c100_y_train_orig, c100_num_classes)
c100_y_test = keras.utils.to_categorical(c100_y_test_orig, c100_num_classes)

print(c100_x_train.shape, c100_x_test.shape, c100_y_train.shape, c100_y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 100) (10000, 100)


In [7]:
c100_googlenet = googlenet(c100_input_shape, c100_num_classes)
#c100_googlenet.summary()

In [8]:
c100_googlenet.fit(c100_x_train, c100_y_train, validation_data = (c100_x_test, c100_y_test), epochs = c100_epochs, batch_size = c100_batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 94s 2ms/step - loss: 3.9716 - accuracy: 0.0883 - val_loss: 4.7106 - val_accuracy: 0.0271
Epoch 2/50
50000/50000 [==============================] - 83s 2ms/step - loss: 3.3793 - accuracy: 0.1817 - val_loss: 3.2536 - val_accuracy: 0.20293811 - accura
Epoch 3/50
50000/50000 [==============================] - 83s 2ms/step - loss: 3.0526 - accuracy: 0.2395 - val_loss: 3.0299 - val_accuracy: 0.2425
Epoch 4/50
50000/50000 [==============================] - 83s 2ms/step - loss: 2.7880 - accuracy: 0.2940 - val_loss: 2.8335 - val_accuracy: 0.2820
Epoch 5/50
50000/50000 [==============================] - 83s 2ms/step - loss: 2.5646 - accuracy: 0.3425 - val_loss: 2.5946 - val_accuracy: 0.3344ccu
Epoch 6/50
50000/50000 [==============================] - 83s 2ms/step - loss: 2.3901 - accuracy: 0.3815 - val_loss: 2.5111 - val_accuracy: 0.3537
Epoch 7/50
50000/50000 [============================

In [9]:
scores = c100_googlenet.evaluate(c100_x_test, c100_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 661us/step
Test loss: 2.7607055408477783
Test accuracy: 0.48019999265670776


In [10]:
predictions = {}
for i in range(c100_num_classes):
    index = np.where(c100_y_test_orig == i)
    slice_x_test = c100_x_test[index[0]]
    slice_y_test = c100_y_test[index[0]]
    scores = c100_googlenet.evaluate(slice_x_test, slice_y_test, verbose = 0)
    print('Class: ' + c100_class_name[i] + ' Test accuracy: ' + str(scores[1]))
    predictions[c100_class_name[i]] = str(scores[1])

Class: apple Test accuracy: 0.7200000286102295
Class: aquarium_fish Test accuracy: 0.6899999976158142
Class: baby Test accuracy: 0.4300000071525574
Class: bear Test accuracy: 0.25
Class: beaver Test accuracy: 0.30000001192092896
Class: bed Test accuracy: 0.36000001430511475
Class: bee Test accuracy: 0.5699999928474426
Class: beetle Test accuracy: 0.4300000071525574
Class: bicycle Test accuracy: 0.6899999976158142
Class: bottle Test accuracy: 0.6200000047683716
Class: bowl Test accuracy: 0.20999999344348907
Class: boy Test accuracy: 0.3499999940395355
Class: bridge Test accuracy: 0.47999998927116394
Class: bus Test accuracy: 0.49000000953674316
Class: butterfly Test accuracy: 0.38999998569488525
Class: camel Test accuracy: 0.5099999904632568
Class: can Test accuracy: 0.5
Class: castle Test accuracy: 0.7200000286102295
Class: caterpillar Test accuracy: 0.38999998569488525
Class: cattle Test accuracy: 0.4000000059604645
Class: chair Test accuracy: 0.05000000074505806
Class: chimpanzee Tes

In [11]:
predictions

{'apple': '0.7200000286102295',
 'aquarium_fish': '0.6899999976158142',
 'baby': '0.4300000071525574',
 'bear': '0.25',
 'beaver': '0.30000001192092896',
 'bed': '0.36000001430511475',
 'bee': '0.5699999928474426',
 'beetle': '0.4300000071525574',
 'bicycle': '0.6899999976158142',
 'bottle': '0.6200000047683716',
 'bowl': '0.20999999344348907',
 'boy': '0.3499999940395355',
 'bridge': '0.47999998927116394',
 'bus': '0.49000000953674316',
 'butterfly': '0.38999998569488525',
 'camel': '0.5099999904632568',
 'can': '0.5',
 'castle': '0.7200000286102295',
 'caterpillar': '0.38999998569488525',
 'cattle': '0.4000000059604645',
 'chair': '0.05000000074505806',
 'chimpanzee': '0.6800000071525574',
 'clock': '0.4399999976158142',
 'cloud': '0.5600000023841858',
 'cockroach': '0.699999988079071',
 'couch': '0.5',
 'crab': '0.5099999904632568',
 'crocodile': '0.27000001072883606',
 'cup': '0.5199999809265137',
 'dinosaur': '0.5099999904632568',
 'dolphin': '0.3799999952316284',
 'elephant': '0.